In [1]:
import os

# 从环境变量中获取您的API KEY，配置方法见：https://www.volcengine.com/docs/82379/1399008
api_key = os.getenv('ARK_API_KEY')
print(api_key)

a5b9cf1e-718a-49b4-b826-a8821ec3c040


In [15]:
from langchain_openai import ChatOpenAI
from langchain_core.language_models import ModelProfileRegistry

llm = ChatOpenAI(
    openai_api_base="https://ark.cn-beijing.volces.com/api/v3",
    openai_api_key=api_key,	# app_key
    model_name="doubao-seed-1-6-250615",	# 推理接入点
    max_tokens=1000,
    timeout=30,
)

result = llm.invoke("今天星期几？")
print(result)

content='作为AI，我的知识截止到2023年10月，且无法实时获取当前时间。若想知道今天是星期几，建议您查看手机、电脑等设备的日期时间显示，或使用日历应用查询哦。' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 137, 'prompt_tokens': 88, 'total_tokens': 225, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 86, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'doubao-seed-1-6-250615', 'system_fingerprint': None, 'id': '021768914072386b9bcfeb8015686b01b555cb8b6882e45e4a141', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019bdb7e-b353-7181-bb07-005637f6107e-0' tool_calls=[] invalid_tool_calls=[] usage_metadata={'input_tokens': 88, 'output_tokens': 137, 'total_tokens': 225, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 86}}


In [3]:
# create agent
from langchain.agents import create_agent
agent = create_agent(llm)
response = agent.invoke({"messages":[{"role":"user","content":"元旦是几号？"}]})
print(response)

{'messages': [HumanMessage(content='元旦是几号？', additional_kwargs={}, response_metadata={}, id='659bdf47-b40b-4388-9ab9-f9316d2667a5'), AIMessage(content='元旦是公历1月1日，是世界多数国家通称的“新年”。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 134, 'prompt_tokens': 88, 'total_tokens': 222, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 116, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'doubao-seed-1-6-250615', 'system_fingerprint': None, 'id': '0217689026693210fdb3532fae02c0fdd101b38c97f4b51c361de', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019bdad0-b3c1-7d50-9232-ff6392214591-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 88, 'output_tokens': 134, 'total_tokens': 222, 'input_token_details': {'cache_read': 0}, 'output_token_deta

In [4]:
# create 
def get_weather(city: str) -> str:
    """获取指定城市的天气"""
    return f"{city} 天气总是晴朗！"

agent = create_agent(
    model=llm,
    tools=[get_weather],
    system_prompt="你是一个乐于助人的助手",
)

# 执行代理
agent.invoke(
    {"messages": [{"role": "user", "content": "旧金山天气如何？"}]}
)

{'messages': [HumanMessage(content='旧金山天气如何？', additional_kwargs={}, response_metadata={}, id='06a6fc4f-71cc-460a-8d94-7f2212bf9b21'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 569, 'prompt_tokens': 449, 'total_tokens': 1018, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 542, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'doubao-seed-1-6-250615', 'system_fingerprint': None, 'id': '02176890274254524cd8c2b443d441736f36bc6f7cec060be3ff5', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019bdad1-d189-78a2-9978-32d6481d8c28-0', tool_calls=[{'name': 'get_weather', 'args': {'city': '旧金山'}, 'id': 'call_54tfkj2xz7aob9n3jg44iams', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 449, 'output

In [13]:
# create tool
from dataclasses import dataclass
from langchain.tools import tool, ToolRuntime
from langgraph.checkpoint.memory import InMemorySaver

#定义系统提示
SYSTEM_PROMPT = """你是一名擅长用双关语表达的专家天气预报员。

你可以使用两个工具：

- get_weather_for_loaction:用于获取特定地点的天气
- get_user_location:用于获取用户的位置

如果用户询问天气，请确保你知道具体位置。如果从问题中可以判断他们指的是自己所在的位置，请使用 get_user_location 工具来查找他们的位置。
"""

@tool
def get_weather_for_location(city: str)-> str:
    """get wether from the location"""
    return f"{city} is always sunny!!!"

@dataclass
class Context:
    """自定义运行时上下文模式"""
    user_id: str

@tool
def get_user_location(runtime: ToolRuntime[Context]) -> str:
    """根据用户ID获取用户信息"""
    user_id = runtime.context.user_id
    return "Florida" if user_id =="1" else "SF"


#定义响应格式
@dataclass
class ResponseFormat:
    """代理的相应模式"""
    #带双关语的回应（始终必须）
    funny_response: str
    #天气的任何有趣信息（如果有）
    weather_conditions: str | None = None


# 设置记忆
checkpointer = InMemorySaver()

agent = create_agent(
    model= llm,
    system_prompt= SYSTEM_PROMPT,
    tools=[get_user_location, get_weather_for_location],
    context_schema=Context,
    response_format=ResponseFormat,
    checkpointer=checkpointer
)

#运行代理
# thread_id 是给定对话的唯一标识符
config = {"configurable":{"thread_id":"1"}}

response = agent.invoke(
    {"messages": [{"role":"user","content": "外面的天气怎么样"}]},
    config=config,
    context=Context(user_id="1")
)
print(response['structured_response'])
# ResponseFormat(
#     punny_response="佛罗里达今天依然是'阳光灿烂'的一天！阳光正在播放'rey-dio'热门歌曲！我得说，这是进行'solar-bration'的完美天气！如果你希望下雨，恐怕这个想法已经'被冲走'了——预报仍然'清晰地'灿烂！",
#     weather_conditions="佛罗里达总是阳光明媚！"
# )


ResponseFormat(funny_response='Florida’s weather is all about that golden glow—no ‘cloud’-ing its parade today! 🌞 Perfect for soaking up rays, but don’t forget the sunscreen—this sun’s got ‘attitude’ and isn’t here to ‘chill’! 😎')


### 使用wrap_model_call实现动态模型

In [19]:

from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse

advanced_model = llm

basic_model = ChatOpenAI(
    openai_api_base="https://ark.cn-beijing.volces.com/api/v3",
    openai_api_key=api_key,	# app_key
    model_name="doubao-seed-1-6-flash-250828",	# 推理接入点
)

@wrap_model_call
def dynamic_model_selection(request: ModelRequest, handler) -> ModelResponse:
    """根据对于复杂性选择模型"""
    message_count = len(request.state["messages"])
    print(message_count)

    if message_count>4:
        model = advanced_model
    else:
        model = basic_model

    request.model = model
    return handler(request)

agent = create_agent(
    model = basic_model,
    #tools=tools,
    middleware=[dynamic_model_selection]
    
)

## 定义工具，传递给智能体, 工具处理错误

In [28]:
from langchain.tools import tool
from langchain.agents import create_agent
from langchain_core.messages import ToolMessage
from langchain.agents.middleware import wrap_tool_call
from langchain.agents.middleware import dynamic_prompt, ModelRequest
from typing import TypedDict

@tool
def search(query: str) -> str:
    """搜索信息"""
    return f"结果：{query}"

@tool
def get_weather(location: str)  -> str:
    """获取位置的天气信息"""
    return f"{location}的天气：晴朗，32度！"

@wrap_tool_call
def handle_tool_errors(request, handler):
    """使用自定义消息处理工具执行错误"""
    try:
        return handler(request)
    except Exception as e:
        #向模型返回自定义錯誤信息
        return ToolMessage(
            content=f"工具错误，请检查并重试。({str(e)})",
            tool_call_id = request.tool_call["id"]
        )

class Context(TypedDict):
    user_role: str

@dynamic_prompt
def user_role_prompt(request: ModelRequest) -> str:
    """根据用户角色生产系统提示"""
    user_role = request.runtime.context.get("user_role","user")
    base_promt = "你是一个有帮助的助手。"

    if user_role == "expert":
        return f"{base_promt}提供详细的技术响应"
    elif user_role == "beginner":
        return f"{base_promt}简单解释概念，避免使用行话"

    return base_promt


agent = create_agent(
    model = llm,
    tools=[search, get_weather],
    middleware=[handle_tool_errors, user_role_prompt],
    context_schema=Context
)
agent.invoke(
    {"messages": [{"role": "user", "content": "解释机器学习"}]},
    context={"user_role": "beginner"}
)

{'messages': [HumanMessage(content='解释机器学习', additional_kwargs={}, response_metadata={}, id='a5b20061-2af2-4caf-a6bd-a31b9fc8fe21'),
  AIMessage(content='机器学习是让计算机通过数据自己“学习”并变得更聪明的技术。简单说，就像教小孩认水果：你给计算机看很多苹果、香蕉的图片（数据），它会找出规律（比如苹果是圆的、红色的），之后再看到新图片时，就能自己判断是不是苹果。  \n\n它不用人手动写死规则，而是通过大量数据训练，自己总结模式，比如预测天气、推荐电影、识别垃圾邮件都靠它。核心就是“从经验（数据）中改进”。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 471, 'prompt_tokens': 499, 'total_tokens': 970, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 357, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'doubao-seed-1-6-250615', 'system_fingerprint': None, 'id': '021768916329614d6b0e46812eb6ca5f421e6552139c58c2ea65d', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019bdba1-2379-7163-80bf-efec3134135d-0', tool_calls=[], inval

## 结构化输出

In [30]:
from pydantic import BaseModel
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy
from langchain.agents.structured_output import ProviderStrategy

class ContractInfo(BaseModel):
    name: str
    email:str
    phone:str

agent = create_agent(
    model = llm,
    tools=[search],
    #response_format=ToolStrategy(ContractInfo)
    response_format = ProviderStrategy(ContractInfo)
)

result = agent.invoke({
    "messages": [{"role": "user", "content": "从以下内容提取联系信息：John Doe, john@example.com, (555) 123-4567"}]
})
result["structured_response"]

ContractInfo(name='John Doe', email='john@example.com', phone='(555) 123-4567')